In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2536)

In [15]:
x[:,:2]

matrix([[ 3559.18189461,  3579.6042701 ],
        [ 3460.49638594,  3406.27419421],
        [ 4216.56952174,  4374.53703851],
        [ 4365.28530544,  4418.49159572],
        [ 4397.76494879,  4446.15681854],
        [ 4135.70576372,  4087.35218113],
        [ 3739.01149581,  3680.39350638],
        [ 3556.60777478,  3641.83364389],
        [ 4537.42235617,  4877.20327903],
        [ 4791.34114601,  4666.30814367],
        [ 4084.9817152 ,  4196.65291013],
        [ 3679.9198082 ,  3814.11198208],
        [ 3237.95449346,  3347.71075256],
        [ 3941.282821  ,  3963.06621888],
        [ 3690.83448591,  3586.29128823],
        [ 2721.04283702,  2767.8007412 ],
        [ 3626.15372335,  3585.20593881],
        [ 3634.55878346,  3600.30836366],
        [ 4040.96338604,  4053.44806974],
        [ 3730.77030338,  3885.95797859],
        [ 4009.36162242,  4150.23724302],
        [ 3386.51550601,  3854.56400166],
        [ 3686.31606317,  3608.7063837 ],
        [ 3647.19307872,  3566.639

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+01, 3e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.93359963e+07  0.00000000e+00  5.65e+03 2.01e+03  1.00e+06     0s
   1  -2.16677198e+07 -1.14790593e+07  1.30e+03 4.63e+02  2.89e+05     0s
   2  -1.68770696e+07 -1.82131576e+07  4.98e+01 1.77e+01  4.17e+04     0s
   3  -1.69579654e+07 -1.71373247e+07  8.01e-01 2.84e-01  3.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.124196086811847e-11,
 1985.3170680123103,
 2.3316673592932125e-11,
 5074.872942122316,
 0.0014771638591546676,
 0.014676779791360173,
 7.036245517390779e-09,
 0.00668311605587818,
 0.007225097955477933,
 3516.3256032302284,
 0.0015525314704907256,
 287.3695703224846,
 0.004852842272908053,
 0.004881672071982979,
 3487.0494269446617,
 1.176484031072789,
 0.002468871097298486,
 0.0024654529303172468,
 2274.4173582596945,
 0.014787998000615588,
 0.014513931632962662,
 2126.0968684820787,
 1998.418576004191,
 1.3023617287052094,
 488.9875439029317,
 48.46758907128421,
 2405.421795603327,
 3506.137294333273,
 941.3957591970341,
 0.13193690371966582,
 1.5829409158039738,
 584.1230875906646,
 3480.6711534533424,
 1.0397781242228143,
 1260.09888559464,
 0.00013313125654795983,
 0.00013307525705111376,
 0.00013311456726595224,
 2630.2928116563685,
 0.3192325775245166,
 3549.3631312159673,
 261.25978860680294,
 1142.8554452452317,
 2524.451977236823,
 1.8642107616942383,
 274.3117527765546,
 